# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,gamba,-2.6500,10.0000,22.91,84,59,3.78,GA,1690389111
1,1,la'ie,21.6477,-157.9253,25.98,76,59,6.47,US,1690389111
2,2,port-aux-francais,-49.3500,70.2167,5.30,85,93,19.42,TF,1690389057
3,3,kerikeri,-35.2268,173.9474,9.62,80,51,2.84,NZ,1690389089
4,4,georgetown,5.4112,100.3354,28.96,89,20,1.54,MY,1690388834


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles= "OSM",
    frame_width = 800,
    frame_height= 600,
    color="City",
    size = "Humidity",
    alpha = .70,
    hover_cols = ["Country"])

# Display the map
city_map_plot




:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_down_city_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) \
                                    & (city_data_df["Wind Speed"] < 4.5) \
                                    & (city_data_df["Cloudiness"] == 0)].dropna()
narrowed_down_city_df




,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
17,17,caconda,-13.7333,15.0667,21.69,18,0,1.45,AO,1690389113
39,39,dudinka,69.4058,86.1778,21.46,70,0,3.73,RU,1690389118
156,156,arhavi,41.3492,41.3069,25.95,80,0,0.64,TR,1690389136
201,201,sinop,41.7711,34.8709,23.34,50,0,3.29,TR,1690389083
248,248,agacoren,38.8748,33.9167,24.31,24,0,0.09,TR,1690389152
344,344,temryuk,45.2722,37.3811,25.14,67,0,1.80,RU,1690389175
507,507,sudak,44.8500,34.9667,23.08,71,0,2.38,UA,1690389203
545,545,dryden,49.7833,-92.7503,24.16,73,0,3.09,CA,1690388885


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_down_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
17,caconda,AO,-13.7333,15.0667,18,
39,dudinka,RU,69.4058,86.1778,70,
156,arhavi,TR,41.3492,41.3069,80,
201,sinop,TR,41.7711,34.8709,50,
248,agacoren,TR,38.8748,33.9167,24,
344,temryuk,RU,45.2722,37.3811,67,
507,sudak,UA,44.8500,34.9667,71,
545,dryden,CA,49.7833,-92.7503,73,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 12000
params = {"categories": "accommodation.hotel","apiKey":geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
caconda - nearest hotel: No hotel found
dudinka - nearest hotel: Виктория
arhavi - nearest hotel: Terzioğlu
sinop - nearest hotel: No hotel found
agacoren - nearest hotel: No hotel found
temryuk - nearest hotel: Немецкая слобода
sudak - nearest hotel: Владислав
dryden - nearest hotel: Holiday Inn Express


,City,Country,Lat,Lng,Humidity,Hotel Name
17,caconda,AO,-13.7333,15.0667,18,No hotel found
39,dudinka,RU,69.4058,86.1778,70,Виктория
156,arhavi,TR,41.3492,41.3069,80,Terzioğlu
201,sinop,TR,41.7711,34.8709,50,No hotel found
248,agacoren,TR,38.8748,33.9167,24,No hotel found
344,temryuk,RU,45.2722,37.3811,67,Немецкая слобода
507,sudak,UA,44.8500,34.9667,71,Владислав
545,dryden,CA,49.7833,-92.7503,73,Holiday Inn Express


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles= "OSM",
    frame_width = 800,
    frame_height= 600,
    color="City",
    size = "Humidity",
    alpha = .70,
    hover_cols = ["Hotel Name","Country"])
# Display the map
hotel_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)